In [1]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [3]:
# --- QUANTUM LOOM: HOLOGRAPHIC TELEPORTATION ---
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# 1. SETUP REGISTERS
# 4 Qubits: Q0 (Observer), Q1 (Infalling), Q2 (Black Hole), Q3 (Radiation)
qr = QuantumRegister(4, 'q')
# 4 Classical Bits: 0-1 for Internal Logic, 2-3 for Final Verification
cr = ClassicalRegister(4, 'meas') 
qc = QuantumCircuit(qr, cr)

# 2. CREATE ENTANGLEMENT (The "Wormhole")
# Reference Pair (Observer <-> Infalling)
qc.h(0)
qc.cx(0, 1)

# Black Hole Pair (Interior <-> Radiation)
qc.h(2)
qc.cx(2, 3)

# 3. BLACK HOLE INTERACTION (Bell Measurement)
# Particle Q1 falls into Hole Q2
qc.cx(1, 2)
qc.h(1)
qc.measure(1, 0) # Store in bit 0
qc.measure(2, 1) # Store in bit 1

# 4. HAWKING RADIATION DECODING (Feed-Forward)
# Correct Q3 based on what fell in.
with qc.if_test((cr[0], 1)):
    qc.z(3)
with qc.if_test((cr[1], 1)):
    qc.x(3)

# 5. VERIFY TELEPORTATION
# Check if Q0 (Observer) is now entangled with Q3 (Radiation)
qc.cx(0, 3)
qc.h(0)

# Measure final state into DIFFERENT bits (2 and 3)
qc.measure(0, 2)
qc.measure(3, 3)

# 6. RUN
print(f"Simulating Teleportation on {backend.name}...")
isa_circuits = transpile([qc], backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=4096)
print(f"Job ID: {job.job_id()}")

# 7. DETAILED HOLOGRAPHIC ANALYSIS
print("\n=== BLACK HOLE TELEPORTATION REPORT ===")
print(f"{'METRIC':<25} | {'VALUE':<10} | {'STATUS'}")
print("-" * 65)

# 1. Parse Results
result = job.result()
counts = result[0].data.meas.get_counts()
total_shots = sum(counts.values())

# We focus on the final state of the Observer (Q0) and Radiation (Q3).
# We want the state "00" on the top two bits (b3, b2).
# This corresponds to the Bell State |Phi+>, indicating perfect transfer.

success_shots = 0
noise_shots = 0

for bitstring, count in counts.items():
    # Clean the string (remove spaces if any)
    bits = bitstring.replace(" ", "")
    
    # Extract the Teleportation Bits (The "Output" channel)
    teleport_signal = bits[0:2] 
    
    if teleport_signal == "00":
        success_shots += count
    else:
        noise_shots += count

# 2. Calculate Fidelity
# Fidelity F = P(Success)
fidelity = success_shots / total_shots

# 3. Calculate "Quantum Advantage"
# Classical Limit: Without entanglement, the best guess is 50% (random).
# Quantum Limit: 100% (Perfect Teleportation).
quantum_advantage = (fidelity - 0.5) * 2  # Normalize to 0-1 scale
if quantum_advantage < 0: quantum_advantage = 0

# 4. Print Detailed Metrics
print(f"{'Raw Fidelity':<25} | {fidelity*100:.1f}%      | ", end="")
if fidelity > 0.9: print("[✓] EXCELLENT")
elif fidelity > 0.7: print("[~] GOOD")
else: print("[X] POOR")

print(f"{'Classical Limit':<25} | 50.0%      | [BASELINE]")
print(f"{'Quantum Channel Strength':<25} | {quantum_advantage*100:.1f}%      | [NET LINK]")

# 5. Visualizer
bar_length = 20
filled = int(fidelity * bar_length)
bar = "█" * filled + "░" * (bar_length - filled)
print(f"\nLINK QUALITY: [{bar}]")

print("-" * 65)

# 6. SCIENTIFIC VERDICT
print("VERDICT ON BLACK HOLE INFORMATION PARADOX:")
if fidelity > 0.8:
    print(">> UNITARITY PRESERVED (NO FIREWALL)")
    print("   The simulation confirms that information falling into the Black Hole")
    print("   was successfully retrieved from the Hawking Radiation.")
    print("   The Event Horizon acts as a smooth, traversable bridge (ER=EPR).")
elif fidelity > 0.6:
    print(">> INFORMATION LEAKAGE DETECTED")
    print("   The connection is noisy, but entanglement persists across the horizon.")
else:
    print(">> INFORMATION LOST (FIREWALL DETECTED)")
    print("   The entanglement was severed. The horizon acted as a decoherence barrier.")

print("-" * 65)

Simulating Teleportation on ibm_marrakesh...
Job ID: d686873e4kfs73d2ncbg

=== BLACK HOLE TELEPORTATION REPORT ===
METRIC                    | VALUE      | STATUS
-----------------------------------------------------------------
Raw Fidelity              | 69.3%      | [X] POOR
Classical Limit           | 50.0%      | [BASELINE]
Quantum Channel Strength  | 38.6%      | [NET LINK]

LINK QUALITY: [█████████████░░░░░░░]
-----------------------------------------------------------------
VERDICT ON BLACK HOLE INFORMATION PARADOX:
>> INFORMATION LEAKAGE DETECTED
   The connection is noisy, but entanglement persists across the horizon.
-----------------------------------------------------------------
